In [67]:
import mysql.connector
from nltk.tokenize import sent_tokenize
import import_ipynb
import re# Membuat stemmer pakai sastrawi
import time
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
from MMR import *

# Deklarasi Connection Database

In [68]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db-berita-mca"  
) 

In [69]:
# Membuat stemmer pakai sastrawi

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [70]:
mycursor = mydb.cursor()

mycursor.execute("SELECT content FROM tb_berita")

myresult = mycursor.fetchall()
print(myresult)

df = pd.DataFrame(myresult,columns=['article'])
df

[('Jakarta, CNBC Indonesia - Krisis mainan jelang Natal terjadi di Inggris. Fenomena ini terjadi karena adanya penundaan penurunan barang-barang dari kontainer di pelabuhan Inggris. Bos retail mainan The Entertainer, Gary Grant, memperingatkan penundaan ini akan mengakibatkan kekurangan stok mainan menjelang Natal. Kini stok boneka Barbie dan mainan Paw Patrol, salah satu mainan favorit anak-anak, diperkirakan akan cepat habis.    Grant mengatakan 170 tokonya terlihat "sangat penuh sekarang". Namun dia mengatakan permintaan "akan melebihi ketersediaan" karena tidak ada cukup driver untuk memindahkan stok mainan perusahaan. "Tidak akan pernah ada toko mainan tanpa mainan. Akan ada toko mainan tanpa semua mainan yang biasanya mereka harapkan karena kelangkaan, dan itu sebagian besar karena masalah transportasi dan gudang, bukan karena kekurangan mainan," kata Grant, dikutip dari BBC International, Jumat (26/11/2021). Hal yang sama juga juga terjadi pada retail lain. Thomas O\'Brien, dire

,article
0,"Jakarta, CNBC Indonesia - Krisis mainan jelang..."
1,"Jakarta, CNBC Indonesia - Aset kripto (cryptoc..."
2,"Jakarta, CNBC Indonesia - Internet merupakan s..."
3,"Jakarta, CNBC Indonesia - Asal muasal virus co..."
4,"Jakarta, CNBC Indonesia - Dunia mata uang krip..."
5,"Jakarta, CNBC Indonesia - Staples Center akan ..."
6,"Jakarta, CNBC Indonesia - Tren investasi kript..."


In [71]:
# Fungsi proses stemming kata
def cleanData(sentence):
    #sentence = re.sub('[^A-Za-z0-9 ]+', '', sentence)
    #sentence filter(None, re.split("[.!?", setence))
    ret = []
    sentence = stemmer.stem(sentence)
    for word in sentence.split():
        if not word in stopwords:
            ret.append(word)
    return " ".join(ret)

In [72]:
# Load Stopwords dengan mengambil dari link url repo github
import requests
def load_stopWords():
    url = "https://raw.githubusercontent.com/Wayan123/Sentiment-Analysis/main/stopwordlist.txt"
    ina_stopword = requests.get(url).content
    return ina_stopword.split()

stopwords = load_stopWords()

In [73]:
print(teksPerkata)

['Jakarta, CNBC Indonesia - Satgas Penanganan Covid-19 melaporkan ada tambahan 339 kasus Covid-19 dalam kurun waktu Sabtu (13/11/2021) pukul 12.00 WIB hingga Minggu (14/11/2021) pukul 12.00 WIB.Tambahan itu lebih rendah dibandingkan dengan kemarin yang tercatat 359.', 'Dengan tambahan 339 kasus hari ini, total kasus Covid-19 di RI mencapai 4.250.855.Sementara itu pasien sembuh dari Covid-19 bertambah 503 menjadi 4.098.178.', 'Sedangkan pasien yang meninggal akibat Covid-19 bertambah 15 menjadi 143.659.', 'Secara kumulatif, total kasus aktif saat ini 9.018, turun 179 pada hari ini.']


In [76]:
from nltk.tokenize import sent_tokenize
sentences = []
for iteration, s in enumerate(df['article']):
    tokenizeKalimat = sent_tokenize(s)


    kalimat = []
#     kalimat.clear()
    cleansingTeks = []
    kalimatAsli = {}
    
    for teks in tokenizeKalimat:
        cleanTeks = cleanData(teks)
        kalimat.append(teks)
        cleansingTeks.append(cleanTeks)
        kalimatAsli[cleanTeks] = teks
    
    setClean = set(cleansingTeks)
#     print(setClean)
#     time.sleep(2)
    
    iteration = MMR()
    score = iteration.similarityScore(cleansingTeks)
    summarySet = iteration.calculateMMR(score,cleansingTeks)
    print(summarySet)
#     print ('\nSummary (hasil teks yang diringkas):\n')
#     for sentence in summarySet:
#         print (kalimatAsli [sentence].lstrip(' '))
#     print ('=============================================================')
#     print ('\nOriginal Passages (Teks Asli):\n')
#     from termcolor import colored # tools untuk membuat highlight merah

#     for sentence in cleansingTeks:
#         if sentence in summarySet:
#             print (colored(kalimatAsli[sentence].lstrip(' '),'red'))
#         else:
#             print (kalimatAsli[sentence].lstrip(' '))


['thomas o brien direktur laksana desainer main boxer gifts yang bas di leeds yang produksi produk di china kata ada banyak stok tetapi jadi sulit karena tunda ini', 'ia saran langgan untuk mulai proses belanja normal mereka lebih awal', 'kini stok boneka barbie dan main paw patrol salah satu main favorit anak kira akan cepat habis', 'labuh komersial besar di inggris felixstowe kata kepada bbc bahwa saat ini ada 50 000 kontainer yang tunggu untuk kumpul karena kurang kemudi truk hgv', 'tidak akan pernah ada toko main tanpa main']
['jakarta cnbc indonesia - aset kripto cryptocurrency alami pekan yang berat telah mayoritas koin kapital pasar besar big cap ambles iring ada kabar buruk lagi untuk bitcoin yang datang dari china dan sentimen negatif lain', 'dalam pek ini kripto wax waxp jadi jawara dengan lonjak 52 93', 'ubah nama itu akan laku pada hari natal saat lakers jamu brooklyn nets', 'kemudian koin kripto besar dua ethereum juga anjlok 7 72 dalam minggu dan coba pulih dengan naik 3 